In [ ]:
from datetime import datetime
import os
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9, 7)

from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import lsst.daf.persistence as dafPersist

In [ ]:
from pfs.utils.fiberids import FiberIds

In [ ]:
base='/projects/HSC/PFS/Subaru'
calibRoot = "/projects/HSC/PFS/Subaru/CALIB-20220630"
rerun='hassans/dev/pipe2d-1069-20220812a'

dataDir = os.path.join(base, "rerun", rerun)
butler = dafPersist.Butler(dataDir, calibRoot=calibRoot)

In [ ]:
fiberIds = FiberIds()

In [ ]:
def computeRaw(flux, refFlux):
    return flux

def computeScaled(flux, refFlux):
    return flux/np.median(flux)

def computeDiff(flux, refFlux):
    return flux - refFlux

def computeScaled(flux, refFlux):
    diffFlux = (flux-refFlux)/np.median(refFlux)
    return diffFlux

In [ ]:
def getData(visit, fiberIds=fiberIds):
    """Get the x/y position [PFI_mm], flux and inR from raw/processed data
    """
    dataId = dict(visit=visit, arm="r", spectrograph=1)
    pfsArm = butler.get('pfsArm', dataId)

    flux = np.empty_like(pfsArm.fiberId, dtype=np.float32)
    for fb in pfsArm.fiberId:
        flux[pfsArm.fiberId==fb] = np.nansum(pfsArm.flux[pfsArm.fiberId==fb])
    
    xyPos=fiberIds.xyForCobras(pfsArm.fiberId)
    
    # Get instrument rotator angle
    exp = butler.get('raw', dataId)
    md = exp.getMetadata()
    inR = md['INSROT']    

    return xyPos, flux, pfsArm.fiberId, inR

def plot(visits, refVisit, nx, ny, compute):
    _, refFlux, _, _ = getData(refVisit)
    fig, _ = plt.subplots(nx, ny, sharex=True, sharey=True) # Don't use axs as fig.axes used later as list
    for visit, ax in zip(visits, fig.axes):
        xyPos, flux, fiberId, inR = getData(visit)
        intensity = compute(flux, refFlux)
        
        sc = ax.scatter('x','y', c=intensity, data=xyPos)
        ax.set_title(f'visit={visit}, InR={round(inR,2)}')
        ax.title.set_fontsize(8)
        ax.set_box_aspect(1)
        plt.colorbar(sc, ax=ax)
#     for ax in fig.axes[len(visits):]:
#         ax.set_visible(False)

In [ ]:
plot([76389, 76390, 76391,76392, 76393, 76394, 76395, 76396, 76398], refVisit=76398, nx=3, ny=3, compute=computeRaw)

In [ ]:
plot([76389, 76390, 76391,76392, 76393, 76394, 76395, 76396, 76398], refVisit=76398, nx=3, ny=3, compute=computeDiff)

In [ ]:
plot([76389, 76390, 76391,76392, 76393, 76394, 76395, 76396, 76398], refVisit=76398, nx=3, ny=3, compute=computeScaled)

In [ ]:
plot([68424, 68425, 68426, 68427], refVisit=68426, nx=2, ny=2, compute=computeScaled)